# Fussy.jl

A Fusion Systems Code

---------

GUI includes (a) Comparisons and (b) Sensitivity Studies for:

 Reactor | | | Affiliation | | | Info
-|-|-|-|-|-|-
Scylla | | | -  | | | Steady-State Prototype
Charybdis | | | - | | | Pulsed Prototype
ARC | | | MIT | | | Compact Steady-State Tokamak
ACT I | | | ARIES (US) | | | Optimistic Design
ACT II | | | ARIES (US) | | | Conservative Design
DEMO Steady | | | PROCESS (EU) | | | Steady-State Machine
DEMO Pulsed | | | PROCESS (EU) | | | Pulsed Machine

<br>
<hr>
<span class="js-load-message cs-loading">// loading should only take a minute.</span>

In [ ]:
cur_script = """
    <style class="js-load-remove-tag">
        .cs-loading:after {
            overflow: hidden;
            display: inline-block;
            vertical-align: bottom;
            -webkit-animation: cs-ellipsis steps(3,end) 3250ms infinite;      
            animation: cs-ellipsis steps(3,end) 3250ms infinite;
            content: "\\2026";
            width: 0px;
        }
        @keyframes cs-ellipsis { to { width: 0.9em; } }
        @-webkit-keyframes cs-ellipsis { to { width: 0.9em; } }
    </style>

    <script class="js-load-remove-tag">
      \$(".js-load-remove-tag").parent(".output_subarea").css("padding", "0");
    </script>
"""

display(HTML(cur_script))

In [ ]:
using Plots
using Interact
using StringCases
using DataStructures
using FileIO
using JLD2

gr()

return

In [ ]:
cur_decks = [
    :scylla, # pulsed
    :charybdis, # steady state
    :arc, :act_1, :act_2,
    :demo_steady, :demo_pulsed
]

cur_points = 41
min_temp = 0
max_temp = 40

cur_temps = linspace(min_temp, max_temp, cur_points)

cur_params = [ 
    :H, :Q, :epsilon, :kappa_95, :delta_95,
    :nu_n, :nu_T, :l_i, :N_G, :f_D, :Z_eff,
    :max_beta_N, :max_q_95, :max_P_W, :eta_CD,
    :B_CS, :tau_FT, :wave_theta
]

return

In [ ]:
cur_fields = [ 
    "T_bar", "n_bar", "I_P", "R_0", "B_0", "R_CS",
    "P_F", "f_IN", "f_BS", "f_CD", "W_M", "cost", "eta_CD",
    "norm_beta_N", "norm_q_95", "norm_P_E", "norm_P_W", "b", "c", "d"
]

sort!(cur_fields)

cur_xvars = deepcopy(cur_fields)
cur_yvars = deepcopy(cur_fields)

filter!(cur_var -> cur_var != "B_0", cur_xvars)
filter!(cur_var -> cur_var != "R_0", cur_yvars)

unshift!(cur_xvars, "B_0")
unshift!(cur_yvars, "R_0")

cur_fields = cur_yvars

return

In [ ]:
using Fussy

cur_file = "data/comparisons.jld2"
cur_scans = load(cur_file, "cur_scans")

cur_file = "data/sensitivities.jld2"
cur_studies = load(cur_file, "cur_studies")

return

In [ ]:
cur_solutions = Dict()

for cur_deck in cur_decks
    tmp_symbol = Symbol("$(cur_deck)_solution")
    isdefined(Fussy, tmp_symbol) || continue
    cur_solutions[cur_deck] = getfield(Fussy, tmp_symbol)()
end

return

In [ ]:
function comparisons(xvar, yvar, deck, showbad)
    plot()

    cur_modes = Dict(
        "" => :solid
    )

    if startswith(string(deck), "demo")
        cur_modes["_simple"] = :dot
    else
        cur_modes["_steady"] = :dot
    end

    max_x = 0
    max_y = 0

    for (cur_super_key, cur_super_value) in cur_modes

        ( cur_super_key != "_steady" && ( xvar == "eta_CD" || yvar == "eta_CD" ) ) && continue

        tmp_scan = cur_scans[Symbol("$(deck)$(cur_super_key)")]

        tmp_dict = Dict(
            "beta" => tmp_scan.beta_reactors,
            "kink" => tmp_scan.kink_reactors,
            "wall" => tmp_scan.wall_reactors
        )

        for (cur_key, cur_value) in tmp_dict
            max_branches = maximum(map(tmp_reactor -> tmp_reactor.branch_id, cur_value))

            for cur_branch in 1:max_branches
                tmp_reactors = filter(tmp_value -> tmp_value.branch_id == cur_branch, cur_value)

                filter!(tmp_reactor -> tmp_reactor.B_0 < 40, tmp_reactors)
                filter!(tmp_reactor -> tmp_reactor.R_0 < 40, tmp_reactors)

                tmp_v = map(tmp_reactor -> tmp_reactor.is_valid, tmp_reactors)
                ( ( showbad ? length(tmp_reactors) : count(tmp_v) ) > 2 ) || continue

                tmp_x = map(tmp_reactor -> getfield(tmp_reactor, Symbol(xvar)), tmp_reactors)
                tmp_y = map(tmp_reactor -> getfield(tmp_reactor, Symbol(yvar)), tmp_reactors)

                tmp_t = map(tmp_reactor -> tmp_reactor.T_bar, tmp_reactors)
                tmp_i = map(tmp_reactor -> tmp_reactor.I_P, tmp_reactors)

                Fussy.sort_lists!(tmp_t, tmp_v, tmp_x, tmp_y, tmp_i)

                if showbad
                    first_index = 1
                    last_index = length(tmp_reactors)
                else
                    first_index = minimum(find(tmp_v))
                    last_index = maximum(find(tmp_v))
                end


                ( first_index == 1 ) || ( first_index -= 1 )
                ( last_index == length(tmp_reactors) ) || ( last_index += 1 )

                tmp_x = tmp_x[first_index:last_index]
                tmp_y = tmp_y[first_index:last_index]

                tmp_v = tmp_v[first_index:last_index]

                max_x = max(max_x, maximum(tmp_x))
                max_y = max(max_y, maximum(tmp_y))

                cur_prefix = cur_super_key == "" ? "Pulsed" : ( cur_super_key == "_steady" ) ? "Steady" : "Simple"
                cur_label = "$(cur_prefix) -- $(capitalize(cur_key))"
                ( max_branches > 1 ) && ( cur_label *= " ($(Fussy.romanize(cur_branch)))" )

                plot!(tmp_x, tmp_y, label=cur_label, style=cur_super_value)
            end
        end

    end

    min_x = 0
    min_y = 0

    max_x *= 1.4
    max_y *= 1.4

    ( xvar == "P_F" ) || 
        ( max_x = min(30, max_x) )

    ( yvar == "P_F" ) || 
        ( max_y = min(30, max_y) )

    ( xvar == "W_M" ) && ( max_x = 150 )
    ( yvar == "W_M" ) && ( max_y = 150 )

    ( xvar == "R_0" || xvar == "B_0" ) && ( max_x = 30 )
    ( yvar == "R_0" || yvar == "B_0" ) && ( max_y = 30 )

    if startswith(xvar, "norm_")
        max_x = 2.0 
    elseif startswith(xvar, "f_")
        max_x = 1.0
    elseif xvar == "cost"
        max_x = 0.1
    end

    if startswith(yvar, "norm_")
        max_y = 2.0 
    elseif startswith(yvar, "f_")
        max_y = 1.0
    elseif yvar == "cost"
        max_y = 0.1
    end

    if haskey(cur_solutions, deck)
        cur_soln = cur_solutions[deck]
        cur_soln_x = getfield(cur_soln, Symbol(xvar))
        cur_soln_y = getfield(cur_soln, Symbol(yvar))

        ( cur_soln_x != nothing ) &&
            plot!([cur_soln_x, cur_soln_x], [min_y, max_y], color=:black, opacity=0.3, label="", style=:dash)

        ( cur_soln_y != nothing ) &&
            plot!([min_x, max_x], [cur_soln_y, cur_soln_y], color=:black, opacity=0.3, label="", style=:dash)

        ( cur_soln_x != nothing && cur_soln_y != nothing ) &&
            scatter!([cur_soln_x], [cur_soln_y], color=:black, opacity=0.3, label="")

        plot!([0,0], [0,0], color=:black, opacity=0.3, label="Model")
    end

    xlims!(min_x, max_x)
    ylims!(min_y, max_y)

    xname = xvar
    yname = yvar

    xlabel!(xname)
    ylabel!(yname)

    xname = replace(xname, "eta_CD", "\\eta_{CD}")
    yname = replace(yname, "eta_CD", "\\eta_{CD}")

    xname = replace(xname, "95", "{95}")
    yname = replace(yname, "95", "{95}")

    xname = replace(xname, "beta", "\\beta")
    yname = replace(yname, "beta", "\\beta")

    endswith(xname, "_bar") && ( xname = "\\overline $(split(xname, "_")[1])" )
    endswith(yname, "_bar") && ( yname = "\\overline $(split(yname, "_")[1])" )

    startswith(xname, "norm_") && ( xname = "\\left( $(replace(xname, "norm_", "")) \\right)_{norm}" )
    startswith(yname, "norm_") && ( yname = "\\left( $(replace(yname, "norm_", "")) \\right)_{norm}" )

    startswith(xname, "f_") && ( xname = join(map(cur_string -> "{$(cur_string)}", split(xname, "_")), "_") )
    startswith(yname, "f_") && ( yname = join(map(cur_string -> "{$(cur_string)}", split(yname, "_")), "_") )

    cur_title = join(map(capitalize, split(string(deck), "_")), " ")
    title!(cur_title)

    plot!()
end

return

In [ ]:
function sensitivity(xvar, yvar, deck)
    cur_study = cur_studies[xvar][deck]

    work_dict = OrderedDict()
    
    work_dict["Wall"] = cur_study.wall_reactors
    work_dict["Kink"] = cur_study.kink_reactors
    work_dict["Cost"] = cur_study.cost_reactors
    
    filter!(work_reac -> work_reac.is_valid, work_dict["Wall"])
    filter!(work_reac -> work_reac.is_valid, work_dict["Kink"])
    filter!(work_reac -> work_reac.is_valid, work_dict["Cost"])

    max_yy = 0
    
    plot()
    for (cur_index, (cur_key, cur_value)) in enumerate(work_dict)
        isempty(cur_value) && continue
        cur_xx = map(cur_reactor -> getfield(cur_reactor, Symbol(xvar)), cur_value) 
        cur_yy = map(cur_reactor -> getfield(cur_reactor, Symbol(yvar)), cur_value) 

        max_yy = max(max_yy, maximum(cur_yy))
        
        cur_label = ( cur_key == "Cost" ) ? "Cost" : "Beta-$(cur_key)"
        plot!(cur_xx, cur_yy, label=cur_label, color=cur_index)
    end
    plot!()
    
    cur_bot_value = (1-cur_study.sensitivity)*cur_study.default
    cur_top_value = (1+cur_study.sensitivity)*cur_study.default
    
    plot!(
        [cur_study.default,cur_study.default],
        [0,max_yy*1.25], 
        label="Default", style=:dot, color=4, lineopacity=0.55, width=1.5
    )
    
    plot!(
        [cur_bot_value,cur_bot_value,NaN,cur_top_value,cur_top_value],
        [0,max_yy*1.25,NaN,0,max_yy*1.25], 
        label="Edge", style=:dot, color=5, lineopacity=0.55, width=1.5
    )
    
    max_xx = cur_top_value
    
    xlims!(0, max_xx * 1.25)
    ylims!(0, max_yy * 1.25)

    xlabel!(string(xvar))
    ylabel!(yvar)

    cur_title = join(map(capitalize, split(string(deck), "_")), " ")
    title!(cur_title)
end

return

In [ ]:
function hide_button(cur_index::Int)
    if cur_index < 0 
        cur_string = """
            \$(".jupyter-widgets").closest(".output_area").length
        """
    else
        cur_string = "0"
    end
    
    cur_script = """
        <script class="js-toggle">
            \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).hide()
            \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).next().hide()
            \$(".js-toggle").parent(".output_subarea").css("padding", "0");
            \$(".js-toggle").remove()
        </script>
    """

    display(HTML(cur_script)) 
end

function show_button(cur_index::Int)
    if cur_index < 0 
        cur_string = """
            \$(".jupyter-widgets").closest(".output_area").length
        """
    else
        cur_string = "0"
    end
    
    cur_script = """
        <script class="js-toggle">
            \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).show()
            \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).next().show()
            \$(".js-toggle").parent(".output_subarea").css("padding", "0");
            \$(".js-toggle").remove()
        </script>
    """

    display(HTML(cur_script)) 
end

return

In [ ]:
@manipulate for loading=[true,false]
end

hide_button(0)
return

In [ ]:
@manipulate for Mode=["Comparisons", "Sensitivity Studies"], Χ_Axis in cur_xvars, X_Axis in cur_params, Y_Axis in cur_fields, Deck in cur_decks, Valid=[true,false]
    if Mode == "Comparisons"
        hide_button(-4)
        show_button(-5)
        comparisons(Χ_Axis, Y_Axis, Deck, !Valid)
    else
        hide_button(-5)
        show_button(-4)
        sensitivity(X_Axis, Y_Axis, Deck)
    end
end 

In [ ]:
cur_script = """
    <script class="js-load-remove-tag">
        \$(".js-load-message").remove();
        \$("#appmode-busy").css("opacity","0");
        \$('pre:contains("WebIO: setting up")').parent(".output_subarea").remove();
        \$(".js-load-remove-tag").parent(".output_subarea").remove();
    </script>
"""

display(HTML(cur_script))